In [ ]:
import pandas as pd

pd.set_option('display.max_columns', 50)

In [ ]:
train = pd.read_csv('./data/train.csv')
train.head(3)

In [ ]:
test = pd.read_csv('./data/test.csv')
test.head(3)

In [ ]:
submission = pd.read_csv('./data/gender_submission.csv')
submission.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

y = train['Survived']
X = train[['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']]

In [ ]:
model.fit(X, y)

In [ ]:
columns = ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
X = train[columns]

In [ ]:
model.fit(X, y)

In [ ]:
X.info()

In [ ]:
X['Age'].fillna(X['Age'].mean(), inplace=True)
X.info()

In [ ]:
model.fit(X, y)

In [ ]:
model.predict(test[columns])

In [ ]:
test['Age'].fillna(test['Age'].mean(), inplace=True)

In [ ]:
model.predict(test[columns])

In [ ]:
test[columns].info()

In [ ]:
test['Fare'].fillna(test['Fare'].mean(), inplace=True)

In [ ]:
model.predict(test[columns])

In [ ]:
y_predicted = model.predict(test[columns])

output = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_predicted})
output

In [ ]:
output.to_csv('./submissions/0_naive.csv', index=False)

# Enter Automated Machine Learning

In [ ]:
import azureml.core

print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
df = pd.read_csv('./data/train.csv')

In [ ]:
df.info()

In [ ]:
# Authenticate if we haven't already

from azureml.core.authentication import InteractiveLoginAuthentication
credentials = InteractiveLoginAuthentication()

In [ ]:
# Load the workspace info

from azureml.core import Workspace
ws = Workspace.from_config(auth=credentials)

In [ ]:
# Create a new experiment if we haven't already

from azureml.core import Experiment
exp = Experiment(workspace=ws, name='Clujaggle')

In [ ]:
from azureml.train.automl import AutoMLConfig

In [ ]:
# @future_me, you should probably launch the experiment now ;)
config = AutoMLConfig(
    task='classification',
    primary_metric='AUC_weighted',
    iterations=3,
    training_data=df,
    label_column_name='Survived',
    n_cross_validations=5,
    preprocess=True,
    model_explainability=False
)

In [ ]:
local_run = exp.submit(config, show_output=True)

In [ ]:
best_run, fitted_model = local_run.get_output()
print('Best run:', best_run)
print('-----------')
print('Best model:', fitted_model)

In [ ]:
def generate_submission_file(model, file_name):    
    data = pd.read_csv('./data/test.csv')
    output = pd.DataFrame({
        'PassengerId': test['PassengerId'], 
        'Survived': model.predict(data)
    })
    output.to_csv(file_name, index=False)

In [ ]:
generate_submission_file(fitted_model, './submissions/1_auto.csv')    

## Analyze the trained model

In [ ]:
transformer = fitted_model.steps[0][1]
scaler = fitted_model.steps[1][1]
classifier = fitted_model.steps[2][1]

In [ ]:
type(transformer)

In [ ]:
scaler.model

In [ ]:
classifier

In [ ]:
transformer.get_engineered_feature_names()

In [ ]:
len(transformer.get_engineered_feature_names())

In [ ]:
transformer.get_featurization_summary()

In [ ]:
test = pd.read_csv('./data/test.csv')
test.info()

In [ ]:
test_preprocessed = transformer.transform(test)
test_preprocessed

In [ ]:
pd.DataFrame(test_preprocessed.toarray(), columns=transformer.get_engineered_feature_names())

In [ ]:
test.tail(5)

In [ ]:
test_scaled = scaler.transform(test_preprocessed)
test_scaled

In [ ]:
pd.DataFrame(test_scaled.toarray(), columns=transformer.get_engineered_feature_names())

In [ ]:
results = classifier.predict(test_scaled)
results

## One more try, with more iterations

In [ ]:
config = AutoMLConfig(
    task='classification',
    primary_metric='AUC_weighted',
    iterations=15,
    training_data=df,
    label_column_name='Survived',
    n_cross_validations=5,
    preprocess=True,
    model_explainability=False
)

local_run = exp.submit(config, show_output=True)

In [ ]:
best_run, fitted_model = local_run.get_output()
fitted_model    

In [ ]:
classifier = fitted_model.named_steps['prefittedsoftvotingclassifier']
classifier

In [ ]:
classifier.estimators

In [ ]:
generate_submission_file(fitted_model, './submissions/2_auto_ensemble.csv')

## Explainability

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
config = AutoMLConfig(
    task='classification',
    primary_metric='AUC_weighted',
    iterations=10,
    training_data=df,
    label_column_name='Survived',
    preprocess=True,
    model_explainability=True
)


local_run = exp.submit(config, show_output=True)

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

best_run, fitted_model = local_run.get_output()

client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation(raw=False)

In [ ]:
engineered_explanations.get_feature_importance_dict()

In [ ]:
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=True)

pd.crosstab(df['Sex'], df['Title'])

In [ ]:
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')
df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 
                                            'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

pd.crosstab(df['Sex'], df['Title'])

pd.crosstab(df['Survived'], df['Title'])

In [ ]:
df['Title'].sample(10)

In [ ]:
better_df = df.drop(columns=['PassengerId', 'Name'])

In [ ]:
config = AutoMLConfig(
    task='classification',
    primary_metric='AUC_weighted',
    iterations=15,
    training_data=better_df,
    label_column_name='Survived',
    n_cross_validations=5,
    preprocess=True,
    model_explainability=False
)

local_run = exp.submit(config, show_output=True)

In [ ]:
generate_submission_file(fitted_model, './submissions/3_auto_ensemble_simplified.csv')

# The End

Get the code from [github.com/vladiliescu/talks](https://github.com/vladiliescu/talks)